In [1]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
# 加载模型
loaded_model = XGBClassifier()
loaded_model.load_model('xgboost_model_52_75.json')

In [1]:
# 获取最新数据
import pandas as pd

In [2]:
df = pd.read_csv('../xGboost/all-coin.csv')

In [3]:
df = df[df['tag'] == 2]

In [4]:
df.head()

,Unnamed: 0,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,...,bollinger_upper,bollinger_middle,bollinger_lower,diff_bollinger_upper,diff_bollinger_lower,diff_sma_3,diff_sma_6,diff_sma_12,signal,tag
200854,0,1504272600000,4830.00,4830.00,4780.83,4781.34,0.391574,1504273499999,1873.081796,10,...,4802.485146,4764.2635,4726.041854,-21.145146,55.298146,-3.243333,-3.118333,6.464167,1,2
200855,1,1504273500000,4781.72,4802.14,4777.02,4777.02,2.600095,1504274399999,12458.756753,26,...,4803.622512,4765.6150,4727.607488,-26.602512,49.412512,-1.570000,-6.941667,0.844167,1,2
200856,2,1504274400000,4777.02,4788.00,4740.00,4740.01,7.238128,1504275299999,34485.963353,59,...,4804.022301,4765.3495,4726.676699,-64.012301,13.333301,-26.113333,-37.286667,-33.635000,1,2
200857,3,1504275300000,4740.02,4759.00,4740.00,4740.00,6.729902,1504276199999,31942.116738,20,...,4804.601132,4764.8500,4725.098868,-64.601132,14.901132,-12.343333,-28.463333,-31.472500,1,2
200858,4,1504276200000,4740.00,4748.93,4740.00,4748.93,0.429827,1504277099999,2037.868549,14,...,4803.015908,4765.7460,4728.476092,-54.085908,20.453908,5.950000,-11.855000,-23.379167,1,2


In [5]:
df.tail()

,Unnamed: 0,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,...,bollinger_upper,bollinger_middle,bollinger_lower,diff_bollinger_upper,diff_bollinger_lower,diff_sma_3,diff_sma_6,diff_sma_12,signal,tag
401703,200849,1685545200000,27033.87,27064.03,26925.34,26925.35,597.15912,1685546099999,1.611675e+07,12606,...,27223.720500,27076.4615,26929.202500,-298.370500,-3.852500,-58.263333,-77.535000,-141.9700,1,2
401704,200850,1685546100000,26925.35,27001.30,26925.34,26937.85,563.84125,1685546999999,1.520480e+07,9793,...,27229.117310,27069.3775,26909.637690,-291.267310,28.212310,-27.836667,-35.158333,-116.7900,1,2
401705,200851,1685547000000,26937.85,26970.62,26921.89,26939.07,341.72631,1685547899999,9.207572e+06,8289,...,27231.756301,27062.0320,26892.307699,-292.686301,46.762301,4.980000,-15.115000,-102.6875,1,2
401706,200852,1685547900000,26939.07,26949.86,26880.58,26911.54,542.61341,1685548799999,1.460538e+07,11281,...,27237.393143,27054.7825,26872.171857,-325.853143,39.368143,-17.946667,-45.010000,-116.8175,1,2
401707,200853,1685548800000,26911.55,26962.55,26839.01,26959.86,665.13213,1685549699999,1.789155e+07,13164,...,27235.876783,27048.8535,26861.830217,-276.016783,98.029783,23.036667,8.605000,-51.2625,1,2


In [ ]:
from binance.spot import Spot
import configparser
# 当前时间
import time
import pandas as pd


config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['keys']['api_key']
api_secret = config['keys']['api_secret']


client = Spot(api_key=api_key, api_secret=api_secret)

def feature(df):
    # Calculate RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Calculate EMA
    ema_short = df['close'].ewm(span=12).mean()
    ema_long = df['close'].ewm(span=26).mean()

    # Calculate MACD
    macd = ema_short - ema_long
    signal_line = macd.ewm(span=9).mean()
    histogram = macd - signal_line

    # Add RSI, EMA, and MACD to the DataFrame
    df['rsi'] = rsi
    df['ema_short'] = ema_short
    df['ema_long'] = ema_long
    df['macd'] = macd
    df['signal_line'] = signal_line
    df['histogram'] = histogram

    df['sma_3'] = df['close'].rolling(window=3).mean()
    df['sma_6'] = df['close'].rolling(window=6).mean()
    df['sma_12'] = df['close'].rolling(window=12).mean()
    # 波动率
    df['volatility_std'] = df['close'].rolling(window=5).std()
    df['pct_change'] = df['close'].pct_change()
    df['sma_20'] = df['close'].rolling(window=20).mean()
    df['std_20'] = df['close'].rolling(window=20).std()
    df['bollinger_upper'] = df['sma_20'] + 2 * df['std_20']
    df['bollinger_middle'] = df['sma_20']
    df['bollinger_lower'] = df['sma_20'] - 2 * df['std_20']
    df['diff_bollinger_upper'] = df['close'] - df['bollinger_upper']
    df['diff_bollinger_lower'] = df['close'] - df['bollinger_lower']
    df['diff_sma_3'] = df['close'] - df['sma_3']
    df['diff_sma_6'] = df['close'] - df['sma_6']
    df['diff_sma_12'] = df['close'] - df['sma_12']
    return df

def get_klines_from_time(symbol, interval, start_time, end_time):
    data = []

    klines = client.klines(
        symbol, interval, startTime=start_time, endTime=end_time)
    
    data += klines

    print(f"当前获取数据时间为：{start_time}", sep="\r")
    print(f"当前获取数据时间为：{end_time}", sep="\r")
    print(f"当前获取数据时间为：{len(klines)}", sep="\r")
    print(klines)


    print("crawl successful")
    return data



now = int(time.time() * 1000)

# 计算1,020分钟之前
before_1020_min = now - 60000 * 88 * 15

data = get_klines_from_time('BTCUSDT', '15m', before_1020_min, now)

df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

df = df.astype(float)
df = feature(df)


# 获取最后68行的数据
df = df.tail(68)

# 将数据平铺
df = df.values.flatten()

print(df.shape)